Reference : https://lsjsj92.tistory.com/568

# LightGCN 기반 DKT 문제 분석
- 🦆 개요 (Overview)
- ⚙️ RecBole 라이브러리 설명
- ⚙️ 환경 설정 및 전처리 (Environment Setting & Preprocessing)
- 🎯 LightGCN 학습
- 🎯 예측 및 평가

## 🦆개요 (Overview)

### DKT, RS, MF Review

- 미션 8 내용 참조
- DKT (Deep Knowledge Tracing) : 사용자의 학습 수준을 추적하기 위한 문제로, 주로 문제에 대해 대상자가 풀 수 있을지 여부를 예측하는 문제
- RS (Recommendation System) : 사용자에게 아이템을 추천하는 문제로, 아이템에 대한 사용자의 선호여부, 선호도 (취향) 를 예측. '선호'를 문제를 맞출 가능성으로 보면 DKT 문제로 치환 가능

### GNN (Graph Neural Network)

- 그래프를 분석하기 위한 딥러닝 (DL)의 일종
- 그래프 (G)는 노드(node, vertex) (V) 와 링크(edge, link)(E)로 구성 (G=(V,E))
- 그래프에는 노드와 링크 각각에 feature가 존재
- GNN에서는 복잡한 그래프의 구조를 임배딩을 통한 추상적인 벡터로 표현하며, 이를 위해 Aggrigation기법을 적용
  - CNN ispired : CNN에서 인접 셀의 값을 참조하여 feature를 구하기 위해 Convolution 기법을 사용하듯, GNN에서도 인접 노드의 값을 참조하여 feature를 구하기 위해 Aggrigation 기법을 사용
  - Aggrigation : 인접 노드의 feature를 가중합을 구하는 등의 방식으로 그래프의 데이터를 규합하는 방법. 반복적으로 Aggrigation 연산을 수행함으로서 멀리에 있는 노드를 포함한 네트워크 구조를 추상화 할 수 있음
  - Node Embedding : Aggrigation 을 통해 노드 자체의 feature 뿐만이 아니라 주변의 연결 구조까지 추상화하여 표현
- GNN의 활용 : Node classification, Graph classification, link prediction 등

### GNN 기반 협업 필터링

- 협업 필터링에서 선호 여부를 링크의 존재, 또는 선호도를 링크의 feature로 가정할 수 있음
- 이 경우, 사용자와 아이템이 노드가 되며 이를 연결하는 선호 여부를 링크로 모델링할 수 있음

### GNN을 통한 DKT 문제

- DKT 에서의 문제를 풀 수 있는지 여부의 문제는 binary 문제이며, 이를 링크가 존재하는지 여부를 판별하는 link prediction으로 예측할 수 있음
- 단, MF 방식과 마찬가지로 협업 필터링 접근방법에 기반한 방식이므로 동일하게 시간에 따른 변화가 없음을 전제로 하여 시간 요소를 고려하기 어려움

### 실습 진행 내용

- 환경 설정 및 전처리 : DKT 문제의 데이터를 협업 필터링에 맞는 형태로 변형하는 방법 설명

## ⚙️ RecBole 라이브러리

### [RecBole](https://github.com/RUCAIBox/RecBole)
- PyTorch기반 추천 시스템 알고리즘 통합 라이브러리 ([문서](https://recbole.io/docs/))
- 다양한 추천 시스템 알고리즘 포함
- 여기에서는 라이브러리에 구현되어 있는 [LightGCN](https://github.com/RUCAIBox/RecBole/blob/master/recbole/model/general_recommender/lightgcn.py) 알고리즘 사용

### [파일 형식](https://recbole.io/docs/user_guide/usage/running_new_dataset.html)
- 새로운 데이터를 적용하기 위해서는 Data loader를 구현하거나 라이브러리에서 정의한 형식에 맞춰 데이터 파일을 생성하는 방법이 있음
- 여기에서는 기존 데이터를 변환하여 데이터 파일을 생성하도록 함

데이터 설정 파일 (data.yaml)

```yaml
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
RATING_FIELD: rating
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, rating, timestamp]

user_inter_num_interval: "[0,inf)"
item_inter_num_interval: "[0,inf)"
val_interval:
    rating: "[0,1]"
    timestamp: "[97830000, inf)"
```

- 사용자, 아이템(문제) 간의 링크만 구현
- rating은 정답을 맞출 수 있을지의 유무 만으로 0 또는 1로 설정
- 설정 파일에서 데이터 파일의 seprator 등 설정 가능 (기본적으론 tab)

```
user_id:token	item_id:token	rating:float	timestamp:float
0	12796	1	1584976631
0	12797	1	1584976634
0	12798	1	1584976642
0	12799	1	1584976649
0	12800	1	1584976656
0	12801	1	1584976667
0	12809	0	1585169523
0	12810	1	1585169530
0	12811	1	1585169594
0	12812	1	1585169609
0	12813	1	1585169628
0	12814	1	1585169635
0	12815	1	1585169651
```

## ⚙️ 환경 설정 및 전처리 (Environment Setting & Preprocessing)

### Install module

- Recbole 모듈 설치

In [1]:
!pip install recbole

     |████████████████████████████████| 2.0 MB 22.4 MB/s eta 0:00:01
     |████████████████████████████████| 27.3 MB 77.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 74.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 81.2 MB/s eta 0:00:01
  Using cached Werkzeug-2.1.1-py3-none-any.whl (224 kB)
     |████████████████████████████████| 781 kB 87.6 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 99.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 67.5 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 92.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 15.7 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 99.4 MB/s eta 0:00:01
  Using cached zipp-3.8.0-py3-none-any.whl (5.4 kB)
     |████████████████████████████████| 77 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 89.0 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: sc

### Loading Library
- 위에서 언급한 Recbole 함수를 로드

In [2]:
from logging import getLogger
import os
import json
import pandas as pd
import time, datetime

from recbole.model.general_recommender.lightgcn import LightGCN

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, get_trainer, init_seed, set_color

from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score

import torch

### 데이터 로드

In [3]:
train_data = pd.read_csv('/opt/ml/input/data/train_data.csv')
test_data  = pd.read_csv('/opt/ml/input/data/test_data.csv')

데이터 통합
- GNN에서 학습되지 않은 노드에 대해 계산 할 수 없음
- 기존 데이터셋에서 테스트 데이터셋의 사용자는 학습 데이터셋에 포함되어 있지 않음
- 이에 따라 모든 데이터를 통합하여 사용하도록 함

In [4]:
data = pd.concat([train_data, test_data])

### 데이터 구성

- 데이터는 학습 데이터셋과 테스트 데이터셋으로 구분되어 있다.
- 각 데이터에는 userID, assessmentItemID, testId, answerCode, Timestamp, KnowledgeTag의 정보가 있다.
- 여기서 assessmentItemID는 문제의 고유 ID이며, answerCode는 사용자가 해당 문제의 정답을 맞췄는지 여부로, 맞췄으면 1, 틀렸으면 0으로 표기된다.
- 기본적인 협업 필터링 적용을 위해 본 실습에서는 userID, assessmentItemID, answerCode만을 사용한다.

In [5]:
userid, itemid = list(set(data.userID)), list(set(data.assessmentItemID))
n_user, n_item = len(userid), len(itemid)

print(f"Train dataset")
display(data.head(5))
print(f" Num. Users    : {n_user}")
print(f" Max. UserID   : {max(userid)}")
print(f" Num. Items    : {n_item}")
print(f" Num. Records  : {len(train_data)}")

Train dataset


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


 Num. Users    : 7442
 Max. UserID   : 7441
 Num. Items    : 9454
 Num. Records  : 2266586


### 데이터 전처리

중복 레코드 제거
 - RS 모델에서는 시간에 따른 변화를 고려하지 않기 때문에 최종 성적만을 바탕으로 평가한다.
 - 사용자+문제항목을 Unique key로 하여 최종 레코드만을 보존하고 나머지 제거한다.

In [8]:
data.drop_duplicates(subset = ["userID", "assessmentItemID"],
                     keep = "last", inplace = True)

평가 항목 제거
- 테스트 데이터셋에서 answerCode가 -1인 항목은 최종 평가시 사용되는 항목으로 여기에선 사용할 수 없다.
- 아래 결과에서와 같이 User, Item 수는 변화 없이 총 레코드 수만 변한다.

In [10]:
data_old = data.copy()
n_user_old, n_item_old = n_user, n_item

data  = data[data.answerCode>=0].copy()

userid, itemid = list(set(data.userID)), list(set(data.assessmentItemID))
n_user, n_item = len(userid), len(itemid)

display(data.tail(5))
print(f" Num. Users    : {n_user}->{n_user}")
print(f" Max. UserID   : {max(userid)}")
print(f" Num. Items    : {n_item}->{n_item}")
print(f" Num. Records  : {len(data_old)}->{len(data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


 Num. Users    : 7442->7442
 Max. UserID   : 7441
 Num. Items    : 9454->9454
 Num. Records  : 2476706->2475962


평가 항목 신규 생성
- 남은 테스트 항목 중, 각 사용자별 최종 레코드를 새로운 평가 항목으로 정한다.

In [11]:
eval_data = data.copy()
eval_data.drop_duplicates(subset = ["userID"],
                     keep = "last", inplace = True)
display(eval_data.head(5))
display(eval_data.tail(5))
print(f" Num. Records  : {len(eval_data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
744,0,A080129006,A080000129,0,2020-12-23 03:40:19,2725
1677,1,A090074006,A090000074,1,2020-11-13 02:47:20,2648
1953,2,A050139007,A050000139,0,2020-10-20 11:32:26,428
2786,5,A080138007,A080000138,1,2020-12-11 22:48:28,8431
3707,6,A030145005,A030000145,0,2020-10-26 09:52:14,7817


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260051,7395,A040122004,A040000122,0,2020-09-08 02:05:18,2102
260066,7404,A030111004,A030000111,1,2020-10-13 09:47:31,7636
260081,7416,A050193003,A050000193,0,2020-10-04 02:44:17,10402
260096,7417,A050193003,A050000193,0,2020-09-06 13:08:54,10402
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


 Num. Records  : 7442


평가 항목을 테스트 항목에서 제거한다.

In [12]:
data.drop(index=eval_data.index, inplace=True, errors='ignore')
display(data.tail(5))
print(f" Num. Records  : {len(data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260107,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


 Num. Records  : 2467499


### 데이터 파일 변환
- 기존 데이터 파일을 recbole 데이터 파일로 변환

사용자, 아이템 ID를 index로 변환

In [13]:
userid, itemid = sorted(list(set(data.userID))), sorted(list(set(data.assessmentItemID)))
n_user, n_item = len(userid), len(itemid)

userid_2_index = {v:i        for i,v in enumerate(userid)}
itemid_2_index = {v:i+n_user for i,v in enumerate(itemid)}
id_2_index = dict(userid_2_index, **itemid_2_index)

데이터 설정 파일 내용 정의

In [15]:
yamldata = """
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
RATING_FIELD: rating
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, rating, timestamp]

user_inter_num_interval: "[0,inf)"
item_inter_num_interval: "[0,inf)"
val_interval:
    rating: "[0,1]"
    timestamp: "[97830000, inf)"
"""

학습데이터 저장

In [16]:
outpath = f"dataset/train_data"
outfile = f"dataset/train_data/train_data.inter"
yamlfile = f"train_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(data.userID, data.assessmentItemID, data.answerCode, data.Timestamp):
    uid, iid = id_2_index[user], id_2_index[item]
    tval = int(time.mktime(datetime.datetime.strptime(tstamp, "%Y-%m-%d %H:%M:%S").timetuple()))
    inter_table.append( [uid, iid, max(acode,0), tval] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


평가 데이터 저장

In [17]:
outpath = f"dataset/test_data"
outfile = f"dataset/test_data/test_data.inter"
yamlfile = f"test_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(eval_data.userID, eval_data.assessmentItemID, eval_data.answerCode, eval_data.Timestamp):
    uid, iid = id_2_index[user], id_2_index[item]
    tval = int(time.mktime(datetime.datetime.strptime(tstamp, "%Y-%m-%d %H:%M:%S").timetuple()))
    inter_table.append( [uid, iid, max(acode,0), tval] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


## 🎯 LightGCN 학습

### 로거 생성

In [18]:
logger = getLogger()

### 설정 인스턴스 생성

In [40]:
# configurations initialization
config = Config(model='LightGCN', dataset="train_data", config_file_list=[f'train_data.yaml'])
config['epochs'] = 1
config['show_progress'] = False
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

logger.info(config)

27 Apr 04:33    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIEL

### 데이터 로드

In [41]:
# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

27 Apr 04:33    INFO  train_data
The number of users: 7443
Average actions of users: 331.5639613007256
The number of items: 9455
Average actions of items: 261.00052887666595
The number of inters: 2467499
The sparsity of the dataset: 96.49371322882392%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']
27 Apr 04:33    INFO  [Training]: train_batch_size = [2048] negative sampling: [{'uniform': 1}]
27 Apr 04:33    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


### 모델 인스턴스 생성

In [42]:
# model loading and initialization
init_seed(config['seed'], config['reproducibility'])
model = LightGCN(config, train_data.dataset).to(config['device'])
logger.info(model)

27 Apr 04:33    INFO  LightGCN(
  (user_embedding): Embedding(7443, 64)
  (item_embedding): Embedding(9455, 64)
  (mf_loss): BPRLoss()
  (reg_loss): EmbLoss()
)
Trainable parameters: 1081472


### 모델 학습

In [43]:
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=True, show_progress=config['show_progress']
)

27 Apr 04:34    INFO  epoch 0 training [time: 32.97s, train loss: 356.7059]
27 Apr 04:34    INFO  epoch 0 evaluating [time: 6.48s, valid_score: 0.147800]
27 Apr 04:34    INFO  valid result: 
recall@10 : 0.0173    mrr@10 : 0.1478    ndcg@10 : 0.0768    hit@10 : 0.3175    precision@10 : 0.0748
27 Apr 04:34    INFO  Saving current: saved/LightGCN-Apr-27-2022_04-33-47.pth


### 학습 결과 출력

In [44]:
# model evaluation
test_result = trainer.evaluate(test_data, load_best_model="True", show_progress=config['show_progress'])

logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
logger.info(set_color('test result', 'yellow') + f': {test_result}')

result = {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

print(json.dumps(result, indent=4))

27 Apr 04:34    INFO  Loading model structure and parameters from saved/LightGCN-Apr-27-2022_04-33-47.pth
27 Apr 04:34    INFO  best valid : OrderedDict([('recall@10', 0.0173), ('mrr@10', 0.1478), ('ndcg@10', 0.0768), ('hit@10', 0.3175), ('precision@10', 0.0748)])
27 Apr 04:34    INFO  test result: OrderedDict([('recall@10', 0.0231), ('mrr@10', 0.1754), ('ndcg@10', 0.1103), ('hit@10', 0.3248), ('precision@10', 0.1071)])


{
    "best_valid_score": 0.1478,
    "valid_score_bigger": true,
    "best_valid_result": {
        "recall@10": 0.0173,
        "mrr@10": 0.1478,
        "ndcg@10": 0.0768,
        "hit@10": 0.3175,
        "precision@10": 0.0748
    },
    "test_result": {
        "recall@10": 0.0231,
        "mrr@10": 0.1754,
        "ndcg@10": 0.1103,
        "hit@10": 0.3248,
        "precision@10": 0.1071
    }
}


## 🎯 예측 및 평가

### 테스트 데이터 로드

In [45]:
# configurations initialization
config = Config(model='LightGCN', dataset="test_data", config_file_list=[f'test_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
test_dataset = create_dataset(config)
logger.info(test_dataset)

27 Apr 04:34    INFO  test_data
The number of users: 7443
Average actions of users: 1.0
The number of items: 1498
Average actions of items: 4.97127588510354
The number of inters: 7442
The sparsity of the dataset: 99.93325329468806%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


### 테스트 예측 및 평가

In [46]:
# 성능 측정
a_prob = model.predict(test_dataset).tolist()
a_true = [val for val in test_dataset.inter_feat["rating"]]
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Test data prediction
 - Accuracy = 32.88%
 - ROC-AUC  = 49.91%


## Inference

In [66]:
infer_df = data_old[data_old.answerCode == -1]
infer_df.answerCode = 0.5
infer_df

/tmp/ipykernel_42659/4121924495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infer_df.answerCode = 0.5


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1035,3,A050133008,A050000133,0.5,2020-10-26 13:13:57,5289
1706,4,A070146008,A070000146,0.5,2020-12-27 02:47:54,9080
3023,13,A070111008,A070000111,0.5,2020-12-27 04:35:09,9660
4283,17,A090064006,A090000064,0.5,2020-10-30 05:48:37,2611
4670,26,A060135007,A060000135,0.5,2020-10-23 11:44:18,1422
...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,0.5,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,0.5,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,0.5,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,0.5,2020-09-06 13:09:15,10402


In [67]:
outpath = f"dataset/inference_data"
outfile = f"dataset/inference_data/inference_data.inter"
yamlfile = f"inference_data.yaml"

os.makedirs(outpath, exist_ok=True)

print("Processing Start")
inter_table = []
for user, item, acode, tstamp in zip(infer_df.userID, infer_df.assessmentItemID, infer_df.answerCode, infer_df.Timestamp):
    uid, iid = id_2_index[user], id_2_index[item]
    tval = int(time.mktime(datetime.datetime.strptime(tstamp, "%Y-%m-%d %H:%M:%S").timetuple()))
    inter_table.append( [uid, iid, max(acode,0), tval] )

print("Processing Complete")

print("Dump Start")
# 데이터 설정 파일 저장
with open(yamlfile, "w") as f:
    f.write(yamldata) 

# 데이터 파일 저장
with open(outfile, "w") as f:
    # write header
    f.write("user_id:token\titem_id:token\trating:float\ttimestamp:float\n")
    for row in inter_table:
        f.write("\t".join([str(x) for x in row])+"\n")

print("Dump Complete")

Processing Start
Processing Complete
Dump Start
Dump Complete


In [68]:
# configurations initialization
config = Config(model='LightGCN', dataset="inference_data", config_file_list=[f'inference_data.yaml'])
config['epochs'] = 1
init_seed(config['seed'], config['reproducibility'])
# logger initialization
init_logger(config)

# dataset filtering
infer_dataset = create_dataset(config)
logger.info(infer_dataset)

27 Apr 04:42    INFO  inference_data
The number of users: 745
Average actions of users: 1.0
The number of items: 445
Average actions of items: 1.6756756756756757
The number of inters: 744
The sparsity of the dataset: 99.77558253525375%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


In [69]:
# 성능 측정
a_prob = model.predict(infer_dataset).tolist()
#a_true = [val for val in infer_dataset.inter_feat["rating"]]
a_pred = [round(v) for v in a_prob] 

#print("Test data prediction")
#print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
#print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

In [77]:
max_v = max(a_prob)
max_v

5.834097862243652

In [78]:
min_v = min(a_prob)
min_v

-2.2850594520568848

In [80]:
result = []
for x in a_prob:
    result.append((x - min_v) / (max_v - min_v))


In [84]:
submit = pd.read_csv('/opt/ml/input/data/sample_submission.csv')
submit

,id,prediction
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
739,739,0.5
740,740,0.5
741,741,0.5
742,742,0.5


In [85]:
submit.prediction = result

In [89]:
submit.to_csv('recbole_lightgcn.csv')